#### Feature selection. This notebook was to remove 1) Non-CpG 2) Non-CpG, X and Y chromosome probes all together  and creating a balanced training and test dataset based on case/control and male/female. In this notebook we will use only covariate adjusted data.

#### Note: It use Kbest approach with Elastic net (L1 and L2 penalty)

#### 1. Import settings

In [ ]:
# Settings imported from other notebook Settings.ipynb
%run Settings.ipynb

In [ ]:
from pyarrow import feather
import pandas as pd
import numpy as np
import os, datetime
from makedirectory import make_directory

In [ ]:
# check if the program is running locally or cluster
process = os.popen('hostname') # open process
p_loc = process.read() 
p_loc = p_loc.strip('\n')

process.close() # close

if p_loc == 'WS-IDRB-404B':
    print("Running locally")
else:
    print("Running on cluster")
    

#### 2. Load data

In [ ]:
# We'll use only covariate adjusted data because we saw the performance was
# better on adjusted data. 
# Now read the imputed data adjusted for covariates
# The adjustment was done on m values and then converted back to beta values
if p_loc == 'WS-IDRB-404B':
    adjusted_beta = feather.read_feather("G:/PGC ML/Covariate Adjusted/2022-03-09_20-58-45/Imputed_Covariate_adjusted_Meth_on_mvals_wo_Neu.feather") 
else:
    adjusted_beta = feather.read_feather("/work/a/ahwani/PGCML/Covariate Adjusted/2022-03-09_20-58-45/Imputed_Covariate_adjusted_Meth_on_mvals_wo_Neu.feather")


In [ ]:
print("Adjusted data frame:")
print(adjusted_beta.iloc[:,0:5].head())

In [ ]:
print("Shape adjusted data {}".format(adjusted_beta.shape))

In [ ]:
# convert cpg names to index
adjusted_beta = adjusted_beta.set_index("Index")

In [ ]:
adjusted_beta.index.names = ["Basename"]

In [ ]:
print(adjusted_beta.iloc[:,0:5].head())

In [ ]:
print("Shape after converting sample ids to row ids {}".format(
    adjusted_beta.shape))

#### Now lets remove non-CpG and X/Y chromosomes

In [ ]:
# get only CpGs
adj_beta_wo_NonCpGs = adjusted_beta.loc[:, adjusted_beta.columns.str.startswith('cg')]


In [ ]:
print("Shape after removing non-CpG probes {}".format(adj_beta_wo_NonCpGs.shape))

In [ ]:
# non_CpG specific probes removed
non_cpg_probes = adjusted_beta.shape[1] - adj_beta_wo_NonCpGs.shape[1]
print("# of non-CpG probes removed: \n", non_cpg_probes)

In [ ]:
# Annotation file

if p_loc == 'WS-IDRB-404B':
    manifest = pd.read_csv("G:/EWAS meta-analysis(Janelle)/data/infinium-methylationepic-v-1-0-b5-manifest-file.csv",
                           skiprows=7, low_memory=False)
else:
    manifest = pd.read_csv("/home/a/ahwani/PGCML/infinium-methylationepic-v-1-0-b5-manifest-file.csv",
                           skiprows=7, low_memory=False)
   

In [ ]:
# print a few of two columns
print("Annotation file head: \n {}".format(manifest[['IlmnID', 'CHR']].head(8)))

In [ ]:
# Lets get X and Y chromosome CpGs
x_cpgs, y_cpgs = [manifest.loc[manifest['CHR'] == cpgs,:] for cpgs in ['X', 'Y']]

In [ ]:
# check if we have got only X and Y chrosome cpgs
print("X and Y chromosome CpGs: \n{}".format(
    [x['CHR'].value_counts() for x in [x_cpgs, y_cpgs]]))

In [ ]:
# combine X and Y chromosomes
XY_cpgs = x_cpgs['IlmnID'].tolist() + y_cpgs['IlmnID'].tolist()
print("# of X and Y probes in manifest :", len(XY_cpgs))

In [ ]:
# Now remove the cpgs from dataframe
adj_beta_wo_NonCpGXY = adj_beta_wo_NonCpGs.drop(columns=XY_cpgs, errors='ignore')

In [ ]:
print("Total probes removed i.e non-CpG, X and Y probes:\n",
       adjusted_beta.shape[1] - adj_beta_wo_NonCpGXY.shape[1])

In [ ]:
print("Shape after removing non-CpG, X and Y probes:\n",
      adj_beta_wo_NonCpGXY.shape)

In [ ]:
# Now make a dict of dfs
adj_df_ls = {'adj_beta_wo_NonCpG':adj_beta_wo_NonCpGs, 
            'adj_beta_wo_NonCpGXY': adj_beta_wo_NonCpGXY}

In [ ]:
print("Remaining CpGs without: \n non-CpGs probes, non-CpGs, X chr and Y chr \n{}".format(
    [x.shape for x in adj_df_ls.values()]))

In [ ]:
# Now check if we removed X and Y chromosome cpgs
def how_many_cpgs(dfs, cpgs):
    """
    Function to check the CpGs present
    """
    print(dfs.keys())
#     print("Key: {} Value {}".format({key: value for (key, value) in dfs}))
    print([x.columns.isin(cpgs).sum() for x in dfs.values()])

In [ ]:
how_many_cpgs(adj_df_ls, XY_cpgs)

In [ ]:
adj_df_ls = {key:value.iloc[:, -1000:] for key, value in adj_df_ls.items()}

In [ ]:
print("adjusted_beta shape:", adj_df_ls['adj_beta_wo_NonCpG'].shape)

In [ ]:
# Check if samples in both files match
(adj_df_ls['adj_beta_wo_NonCpG'].index == adj_df_ls['adj_beta_wo_NonCpGXY'].index).all()

In [ ]:
# Check if samples in both files match
# (adj_df_ls['adjusted_beta'].index == adj_df_ls['adjusted_beta_wo_Y'].index).all()

In [ ]:
# Now load phenotype data
# 2021-07-12_10-28-46
import pandas as pd
if p_loc == 'WS-IDRB-404B':
    pheno = pd.read_csv("G:/PGC ML/Pre_Processed Data/2021-11-15_21-41-53/DNHS_GTP_MRS_ArmyS_Prismo_Pheno.csv")
else:
    pheno = pd.read_csv("/home/a/ahwani/PGCML/DNHS_GTP_MRS_ArmyS_Prismo_Pheno.csv")



In [ ]:
print("Number of males-1 and females-2:\n {}".format(
    pheno["Gender"].value_counts()))


In [ ]:
# Get only those that are in both
pheno = pheno[pheno["Basename"].isin(adjusted_beta.index)]
pheno

In [ ]:
# Now check if samples in pheno and methylation data are in order
(pheno["Basename"] == adj_df_ls['adj_beta_wo_NonCpG'].index).all()

In [ ]:
# Now sort the phenotype data according to methylation data
pheno = pheno.set_index(["Basename"])
pheno = pheno.reindex(index = adjusted_beta.index)
pheno = pheno.reset_index()

In [ ]:
# Check the order again
[(pheno["Basename"] == adj_df_ls[key].index).all() for key in adj_df_ls.keys()]

In [ ]:
pheno

In [ ]:
# In this phenotype file race column has strings like 1,5/2,5
# So we need to remve the substring after , otherwise an error in ML mode
pheno['Race'] = pheno['Race'].str.split(',').str[0]


In [ ]:
pheno['Race'].str.contains(',').any()

In [ ]:
[x.shape for x in adj_df_ls.values()]

In [ ]:
pheno.shape

In [ ]:
# outpheno type file has some columns not needed in ML
# Lets remove them
# Basename we will remove later, because we need it
# pheno = pheno.drop(['Unnamed: 0'], axis=1)

In [ ]:
# count nas values in each column
len(pheno) - pheno.count()

In [ ]:
# As some ptsdlife variable has missing values
# We will remove them and use that a phenotype file 
# when using ptsdlife as outcome variable
pheno_no_na = pheno.dropna()
pheno_ptsdlife = pheno.dropna(subset=['Age', 'Ptsdlife', 'Race'])
pheno_ptsdpm = pheno.dropna(subset=['Age', 'Race' ,'Ptsdpm', 'Traumanum']) # Rows with Nas

In [ ]:
print(pheno_no_na.shape)
print(pheno_ptsdlife.shape)
print(pheno_ptsdpm.shape)


In [ ]:
pheno_ptsdlife.isnull().sum()

In [ ]:
pheno_ptsdpm.isnull().sum()

In [ ]:
def convert_to_int(df, cols):
    """
    Function to convert float to int
    Parameters:
    df: dataframe
    cols: columns names that need conversion
    """
#     df[cols] = df[cols].astype(float)
    df = df.copy()
    df[cols] = df[cols].astype(int)
    return(df)
    
ptsdpm_cols = ['Age', 'Ptsdpm', 'Traumanum']
ptsdlife_cols = ['Age', 'Ptsdlife', 'Traumanum'] 

pheno_ptsdpm = convert_to_int(df = pheno_ptsdpm, cols=ptsdpm_cols)
pheno_ptsdlife = convert_to_int(df = pheno_ptsdlife, cols=ptsdlife_cols)

In [ ]:
pheno_ptsdpm.head()

In [ ]:
pheno_ptsdlife.head()

In [ ]:
# count nas values in each column for ptsdpm and ptsdlife
[len(x) - x.count() for x in [pheno_ptsdpm, pheno_ptsdlife]]

In [ ]:
# pheno_ptsdlife


In [ ]:
# pheno_ptsdpm

In [ ]:
# control and case count
def get_count(vals):
    c = vals.value_counts()
    print("Count ptsd: \n{}".format(c))
    print("Percentage ptsd: \n{}".format(c/c.sum()))
    
get_count(vals = pheno_ptsdpm['Ptsdpm'])


In [ ]:
get_count(vals = pheno_ptsdlife['Ptsdlife'])

In [ ]:
# Check if any missing values 
[k.isna().sum() for k in [pheno_ptsdpm['Ptsdpm'], 
                          pheno_ptsdlife['Ptsdlife'],
                          pheno_ptsdpm['Pts_Severity']]]

In [ ]:
# Here we will fill the missing values using column mean
# It has three cohorts (with lifetime ptsd), and we use individual cohort 
# to impute the missing values
def impute_nas(df, study):
    x = df.loc[df['Study'] == study,:]
    x.fillna(x.mean(), inplace = True)
    return(x)

In [ ]:
pheno_ptsdpm.shape

In [ ]:
print("Samples in each cohort for current ptsd:\n{}".format(
    pheno_ptsdpm["Study"].value_counts()))

In [ ]:
pheno_ptsdlife.shape

In [ ]:
print("Samples in each cohort for lifetime ptsd:\n{}".format(
pheno_ptsdlife["Study"].value_counts()))

In [ ]:
# Call the function to get the data for each study
dfs = [pheno_ptsdpm, pheno_ptsdlife]
study = ['DNHS', 'GTP', 'MRS', "Armystarrs", "Prismo"]
ptsd = [x.loc[x['Study'] == y,:] for x in dfs for y in study]
ptsd = [x.fillna(x.mean()) for x in ptsd]

In [ ]:
[x.shape for x in ptsd]

In [ ]:
# After imputing the missing values, lets combine each study
# First, DNHS,GTP and MRS for ptsdpm (at 0:5 index) and then for ptsdlife(5:10)
pheno_ptsdpm_imp = pd.concat(ptsd[0:5], axis = 0) # all have ptsdpm

In [ ]:
pheno_ptsdpm_imp.shape

In [ ]:
pheno_ptsdlife_imp = pd.concat(ptsd[5:10], axis = 0) # three have ptsdlife

In [ ]:
# Samples from each study after imputation
print("Samples after imputation in each cohort for current ptsd:\n{}".format(
pheno_ptsdpm_imp["Study"].value_counts()))

In [ ]:
print("Samples after imputation in each cohort for lifetime ptsd:\n{}".format(
pheno_ptsdlife_imp["Study"].value_counts()))

In [ ]:
[x.shape for x in dfs]

In [ ]:
[x.shape for x in [pheno_ptsdpm_imp, pheno_ptsdlife_imp]]

In [ ]:
# As case for ptsdpm, we wanted to remove the samples
# with no current ptsd but have lifetime ptsd
flaged_samples = pheno_ptsdpm_imp[(pheno_ptsdpm_imp["Ptsdpm"] == 0) &
                  (pheno_ptsdpm_imp["Ptsdlife"] == 1)]

In [ ]:
print("Flaged samples:",flaged_samples.shape)
print("Flaged samples cohort level:\n{}".format(
    flaged_samples["Study"].value_counts()))

In [ ]:
# Now remove flaged samples
pheno_ptsdpm_imp_wo_rem = pheno_ptsdpm_imp[~pheno_ptsdpm_imp["Basename"].isin(flaged_samples["Basename"])]
pheno_ptsdpm_imp_wo_rem.shape

In [ ]:
# How many are remitted samples
# it should be zero
((pheno_ptsdpm_imp_wo_rem["Ptsdpm"] == 0) &
                  (pheno_ptsdpm_imp_wo_rem["Ptsdlife"] == 1)).sum()

In [ ]:
print(get_count(vals = pheno_ptsdpm_imp_wo_rem['Ptsdpm']))

In [ ]:
[x.isnull().sum() for x in ptsd]

In [ ]:
# Check for missing values
[x.isnull().any().sum() for x in adj_df_ls.values()]

In [ ]:
# check common in ptsdpm and ptsdlife
pheno_ptsdlife_imp["Basename"].isin(pheno_ptsdpm_imp_wo_rem["Basename"]).sum()

In [ ]:
# combine data

# we will prepare three datasets, with response variables
# ptsdpm, ptslife
# We can use the same data for PTSDpm and PTSS
# "ptsdpm_imp",
keys = ["ptsdpm_imp_wo_rem",
       "ptsdlife_imp"]
values = [pheno_ptsdpm_imp_wo_rem,
         pheno_ptsdlife_imp]

def merge_data(df, vals, keys):
    comb = [pd.merge(df, x, 
              left_index=True, 
              right_on= "Basename",
              how='inner') for x in values]
    return(dict(zip(keys, comb)))
    
    

In [ ]:
print(adj_df_ls.keys())

In [ ]:
print("Shape :\n",[(k, adj_df_ls[k].shape) for k in adj_df_ls.keys()])

In [ ]:
# combine methylation data with pheno
combined_dfs, combined_dfs_XY = [merge_data(df = adj_df_ls[key],
                           vals=values, keys=keys) for key in adj_df_ls.keys()]

In [ ]:
print([x.keys() for x in [combined_dfs, combined_dfs_XY]])

In [ ]:
[x.shape for x in combined_dfs.values()]

In [ ]:
[x.shape for x in combined_dfs_XY.values()]

In [ ]:
# outcome variables
# When we consider ptsd, remove PTSS from the predictors
# When using PTSS, remove PTSD
# Labels are same in adjusted/unadjusted data
ptsdlife_labels = np.array(combined_dfs['ptsdlife_imp']['Ptsdlife']) # without MRS
ptsdpm_wo_rem_labels = np.array(combined_dfs["ptsdpm_imp_wo_rem"]["Ptsdpm"])
ptss_labels = np.array(combined_dfs["ptsdpm_imp_wo_rem"]['Pts_Severity'])

In [ ]:
# Check any na in response variables
[np.isnan(x).any() for x in [ptsdlife_labels,
                             ptsdpm_wo_rem_labels,
                             ptss_labels]]

In [ ]:
# Length of outcome variable
[len(x) for x in [ptsdpm_wo_rem_labels,
                  ptsdlife_labels,
                  ptss_labels
                  ]]

In [ ]:
# clear some space
del adjusted_beta, adj_beta_wo_NonCpGs, adj_beta_wo_NonCpGXY

In [ ]:
# remove the columns that we don't need
# keep basename, to know which samples are used in training from ptsdpm
# and use the others samples in testing for ptsdlife 
# because some samples are different in ptsdpm and ptsdlife
rm_cols = ["Unnamed", "Ptsdpm",
           'Ptsdlife', 'Pts_Severity',
           'Study', "Avoidance", 
           "Hyperarousal", "Intrusion",
           "Mdd", "Race"]

rm_set2 = ["Bcell", "Cd4T",  "Cd8T", "Mono",
           "Neu", "Nk", "Smos", "Age", 
           "Comp.2", "Comp.3"] # Gender is removed while training the model

# we are removing these columns for now because
# we want to have methylation score. 
 # "Childhood_Mt", "Traumanum",

all_cols = rm_cols + rm_set2

def drop_columns(df, cols):
    """
    Function to remove columns
    Parameters:
    df: data frame
    cols: columns to remove
    
    """
    df = df.loc[:, ~df.columns.str.contains('|'.join(cols))]
    return(df)
    

In [ ]:
# Now drop the columns we want. As we are using adjusted data now,
# we will drop all_cols
combined_dfs1, combined_dfs2  = [{key: drop_columns(df = value, 
                                  cols= all_cols) for (key, value) in x.items()} 
                                 for x in [combined_dfs, combined_dfs_XY]]

In [ ]:
print([x.keys() for x in [combined_dfs1,combined_dfs2]])

In [ ]:
# combined_dfs1["ptsdpm_imp_wo_rem"]

In [ ]:
combined_dfs2["ptsdpm_imp_wo_rem"].shape

In [ ]:
# check any nas
[x.isnull().sum().any() for x in combined_dfs1.values()]

In [ ]:
[x.isnull().sum().any() for x in combined_dfs2.values()]

In [ ]:
combined_dfs1.keys()

In [ ]:
# creat dictionary of the data and labels
qcd_data = dict({'ptsdpm_wo_NonCpGs':[combined_dfs1["ptsdpm_imp_wo_rem"], ptsdpm_wo_rem_labels],
                 'ptsdlife_wo_NonCpGs':[combined_dfs1["ptsdlife_imp"], ptsdlife_labels],
                 'ptsdpm_wo_NonCpGsXY':[combined_dfs2["ptsdpm_imp_wo_rem"], ptsdpm_wo_rem_labels],
                 'ptsdlife_wo_NonCpGsXY':[combined_dfs2["ptsdlife_imp"], ptsdlife_labels]               
}) 

In [ ]:
print(qcd_data.keys())

In [ ]:
print("Shape of QCd data...\n {}".format([(k, qcd_data[k][0].shape) for k in qcd_data.keys()]))

In [ ]:
# Save qcd data
import nbimporter
import joblib
import os, datetime
from makedirectory import make_directory

In [ ]:
if p_loc == 'WS-IDRB-404B':
    comb_dir = make_directory(maindir="G:/PGC ML/Combined Data", verbose=True)
else:
    comb_dir = make_directory(maindir="/work/a/ahwani/PGCML/Combined_data", verbose=True)



In [ ]:
# save
joblib.dump(qcd_data, os.path.join(comb_dir, "DNHS_GTP_MRS_ArmyS_Prismo_combined.pkl"))


In [ ]:
print("QCd data keys:\n{}".format(qcd_data.keys()))

In [ ]:
print(len(qcd_data["ptsdpm_wo_NonCpGs"][1]))

In [ ]:
# Steps:
# 1. Define covariates
# 2. Split the data into train and test
# 3. From the train split, keep only covariate columns 
# 4. Train model using covariate columns and check performance
# 5. Now replace the outcome variable with the predictions 
#    (expected with observed)
# 6. Drop the covariates from train split, and using the new outcome variable
#    train the models and check the performance 
#    (both, all features and significant)
# 7. Adjust the models if needed
# End

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [ ]:
def get_count(lab, rc = True):
    print(np.unique(lab, return_counts=True))

#### Classification without feature selection

In [ ]:
# We will train the model on current ptsd and test it on current and life ptsd
keys = ['ptsdpm_wo_NonCpGs', 'ptsdpm_wo_NonCpGsXY']
qcd_data_ml = {k:qcd_data[k] for k in keys if k in qcd_data}


In [ ]:
qcd_data_ml.keys()

In [ ]:
# Save models
if p_loc == 'WS-IDRB-404B':
    models_dir = make_directory(maindir="G:/PGC ML/Trained Models",  verbose=True)
else:
    models_dir = make_directory(maindir="/work/a/ahwani/PGCML/models_dir",  verbose=True)



In [ ]:
# joblib.dump(models_bfs, os.path.join(models_dir, "RF_before_fs.pkl"))

In [ ]:
# models_bfs.items()


#### 3. Feature selection functions

In [ ]:
# we will try reducing the features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

class SelectFeatures():
    
    def __init__(self):
        
        """
        Function constructor
        
        """
        self.fit_lis = []
        self.df_imp_lis = []
        
        
    def CallFit(self, df, labels, top_fea):
        """
        Fit model for feature selection. 
        The model will fit the data to find the predictive 
        features for response variable. 
        
        Parameters:
        ----------
        df : List of data frames you want to select features from
        labels : List of response variable. 
        top_fea : Number of features you want to select
        
        """
        top_selected = SelectKBest(score_func = f_classif, k = top_fea)
        fit = top_selected.fit(df, labels)
        self.fit_lis.append(fit)
        
        
    
    def UnivFeatureSelection(self, df, index):
        """
        This function will call the fitted models to select the features
        
        Parameters:
        ----------
        df : List of data frames you want to select features from
        index : Index of the model fitted on the data
        
        """
        cols = self.fit_lis[index].get_support(indices=True)
        df_impt_uvs = df.iloc[:,cols]
        self.df_imp_lis.append(df_impt_uvs)
        return self.df_imp_lis


In [ ]:
# qcd_data_ml['ptsdpm'][0].head()

In [ ]:
from collections import defaultdict

features = np.arange(10, 50, 10)
#features = np.arange(10, 5000, 10)

models_after_fs = defaultdict(list)
accuracy = defaultdict(list)
important_fea = defaultdict(list)
train_test_imp = defaultdict(list)

for i, val in enumerate(features):
    if(val % 10 == 0):
        print("# Features :", val)
    for key in qcd_data_ml:
        print(key)
        FS = SelectFeatures()
        print(features[i])
        
        # remove sample identifier
        df_ml = qcd_data_ml[key][0].loc[:,~qcd_data_ml[key][0].columns.str.contains('|'.join(["Basename",
                                                                                              "Gender"]))]
        FS.CallFit(df = df_ml, 
                   labels = qcd_data_ml[key][1], 
                   top_fea = features[i])
        imp_fea = FS.UnivFeatureSelection(df = df_ml, index = 0 )
        important_fea[key].append((val, imp_fea[0].columns))

        X_train, X_test, y_train, y_test = train_test_split(imp_fea[0], 
                                                            qcd_data_ml[key][1],
                                                            test_size = 0.25,
                                                            random_state = 0,
                                                           stratify=qcd_data_ml[key][1])
        train_test_imp[key].append((val, X_train, X_test, 
                                    y_train, y_test ))

        clf = make_pipeline(MinMaxScaler(), 
                           LogisticRegressionCV(Cs = [0.95],
                                                cv = 5,
                                                solver="saga",
                                                penalty="elasticnet",
                                                l1_ratios=[0.1,0.5],
                                                max_iter=1000,
                                                class_weight = "balanced",
                                                n_jobs = -1))
        clf.fit(X_train, y_train)
        prediction = clf.predict(X_test)
        acc = clf.score(X_test, y_test)
        clf_rep = classification_report(y_test, prediction)
        models_after_fs[key].append((val, clf)) # store #feature and models
        print(clf_rep)
        print("Accuracy :{:.3f}" .format(acc))
        accuracy[key].append((val, acc)) # store #feature and accuracy

In [ ]:
print(train_test_imp.keys())

In [ ]:
# path on cluster
if p_loc != 'WS-IDRB-404B':
    fea_sets_dir = make_directory(maindir="/work/a/ahwani/PGCML/FeatureSets",  verbose=True)
    joblib.dump(important_fea, os.path.join(fea_sets_dir, "Important Feature sets.pkl"))
    joblib.dump(accuracy, os.path.join(fea_sets_dir, "Accuracy_feature_sets.pkl"))


In [ ]:
# write model information
joblib.dump(clf, os.path.join(fea_sets_dir, 'readme.txt'))

In [ ]:
# save models and accuracy after feature selection
# it will be same for local and cluster
model_name = clf.steps[1][0]
joblib.dump(models_after_fs, os.path.join(models_dir, model_name+"_after_fs.pkl"))
joblib.dump(accuracy, os.path.join(models_dir, model_name+"_accuracy_after_fs.pkl"))

In [ ]:
clf